In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time

warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
sns.set(rc={'figure.figsize':(12,6)})

In [10]:
df=pd.read_csv('./US_accidents2.csv')

In [11]:
numerical_cols = ['temperature(f)','humidity(%)','pressure(in)','visibility(mi)','wind_speed(mph)']
#normal distribution - mean is 0 and variance is 1, standard scaling
#scaling
for i in numerical_cols:
    scale = StandardScaler().fit(df[[i]])
    #inplace transformation
    df[i] = scale.transform(df[[i]])

df[['side']] = df[['side']].astype(int) 

In [23]:
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_validate
from sklearn.metrics import classification_report,plot_confusion_matrix,f1_score,accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [44]:
target='severity'
unwanted_col=['Unnamed: 0','id', 'start_time','end_time','description','state','start_lat','start_lng','city','source']

X = df.drop(unwanted_col, axis=1)
y = df[target]
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

In [47]:
y_test.shape

(914450, 5)

In [ ]:
# Build the neural network
model = Sequential()
model.add(Dense(25, input_dim=X.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(5)) # Output
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',restore_best_weights=True)

model.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

In [ ]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, yhat)
print(f'Accuracy: {acc:.3f} %')